In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
import ast
import warnings
import gc

warnings.filterwarnings('ignore') 

pd.set_option("display.max_columns", 100) # ‘None’ value means unlimited.

THEMES = {
 'amitie': 373,
 'amour': 77,
 'autobiographie': 45,
 'aventure': 33,
 'bande-dessinee': 18,
 'biographie': 31,
 'cinema': 89,
 'classique': 28,
 'comedie-romantique': 3788,
 'comics': 142,
 'drogue': 863,
 'dystopie': 879,
 'emotion': 4100,
 'enquetes': 3988,
 'entretiens': 708,
 'essai': 13,
 'famille': 290,
 'fantastique': 7,
 'fantasy': 4,
 'geographie': 187,
 'guerre': 91,
 'humour': 15,
 'humour-noir': 621,
 'jeunesse': 14,
 'journalisme': 975,
 'litterature-americaine': 9,
 'litterature-asiatique': 1064,
 'litterature-francaise': 3,
 'manga': 12,
 'musique': 44,
 'nouvelles': 23,
 'peur': 430,
 'poesie': 25,
 'politique': 54,
 'psychologie': 65,
 'racisme': 906,
 'recit-de-voyage': 448,
 'religion': 26,
 'reportage': 3488,
 'reseaux-sociaux': 26109,
 'roman': 1,
 'roman-fantastique': 912,
 'roman-noir': 136,
 'romans-policiers-et-polars': 63883,
 'science-fiction': 6,
 'sentiments': 1770,
 'serie': 788,
 'theatre': 21,
 'thriller': 11,
 'thriller-psychologique': 1073,
 'tragedie': 601,
 'western': 533
}



# Merging files and datasets

In [2]:
def merge_data_files(books_file_list:list, books_meta_file=None, books_senti_file=None, books_users_file=None):
    df_books = None

    # fusion of mains books data files with comments
    for filename in books_file_list:
        df_books_temp = pd.read_json(filename, lines=True)
        if df_books is None:
            df_books = df_books_temp
        else: df_books = pd.concat([df_books, df_books_temp])

    df_books = df_books.drop(['tags'],axis=1)

    # genre du profil per user_id
    if books_users_file is not None:
        df_users = pd.read_json(books_users_file, lines=True)
        df_books = df_books.merge(df_users, on='user_id', how='left')
    df_books = df_books.fillna('')

    # join to the meta data books file
    if books_meta_file is not None:
        df_meta = pd.read_json(books_meta_file, lines=True)
        df_meta = df_meta.drop(['book_nb_comm', 'title', 'name', 'surname','img_url','book_date'],axis=1)
        df_books = df_books.merge(df_meta, on='book_id', how='inner')

    # join sentiments file
    if books_senti_file is not None:
        df_senti = pd.read_json(books_senti_file, lines=True)
        df_senti = df_senti.drop(['title'],axis=1)
        df_books = df_books.merge(df_senti, on='book_id', how='left')
    df_books = df_books.fillna(0)

    return df_books

In [3]:
# df_comm = merge_data_files(
#     books_file_list = ['../output/books-julien.json','../output/books-rebecca.json'],
#     books_meta_file = '../output/books-meta-data.json',
#     books_senti_list = ['../analyse/df_nlp0_1000.csv','../analyse/df_nlp1000_2000.csv','../analyse/df_nlp2000_3000.csv',
#     '../analyse/df_nlp3000_4000.csv','../analyse/df_nlp3000_4000.csv','../analyse/df_nlp4000_4566.csv'],
#     books_users_file = '../output/users-data.json'
#     )
df_comm = merge_data_files(
    books_file_list = ['../output/books-julien.json','../output/books-rebecca.json'],
    books_meta_file = '../output/books-meta-data.json',
    books_senti_file = '../output/vecteurs_sentiments_final2.json',
    books_users_file = '../output/users-data.json'
    )

In [5]:
# group by per book
columns_senti = [col for col in df_comm.columns if col.startswith('sen_')]
columns_book = ['book_id', 'book_url', 'book_nb_comm', 'title', 'name', 'surname',
       'tags', 'img_url', 'book_rating_count', 'book_rating_value',
       'book_author_url', 'book_editor', 'book_pages', *columns_senti]

def reduce_comm_to_books(df):
    return df.copy().groupby(columns_book, as_index=False).count().loc[:,columns_book]

In [6]:
df_books = reduce_comm_to_books(df_comm)

We have now 2 dataset : df_comm and df_books with 2 differents aggregate levels

# Preprocessing df_books

In [7]:
# on garde que les tags avec le nom dans filter_list ou si la valeur est supérieur à filter_force_min. ca permet de rétirer les tags rares et peu importants
def tags_to_cols(df, col_name, filter_list=None, filter_force_min=24):
    df1 = df.copy()

    for index,row in df.iterrows():
        tags_as_string = row[col_name]
        tags = ast.literal_eval(tags_as_string)

        for tag in tags:
            if filter_list is not None and not tag[0].strip() in filter_list and tag[1] < filter_force_min:
                continue
            tag_name = 'tag_'+tag[0].strip().replace(' ','_').lower()
            df1.loc[df1.index == index, tag_name] = tag[1]

    df1 = df1.fillna(0)
    return df1

In [8]:
df_books = tags_to_cols(df_books, col_name='tags', filter_list=list(THEMES.keys()), filter_force_min=24)

In [9]:
df_books.to_json('../output/final/data-books.json',lines=True,orient='records')
df_comm.to_json('../output/final/data-comm.json',lines=True,orient='records')
df_comm['commentaire'] = ''
df_comm.to_json('../output/final/data-comm-light.json',lines=True,orient='records')

In [69]:
# for api and dashboard
import sqlite3
conn = sqlite3.connect('../output/final/data.db')
df_comm.set_index('book_id').to_sql('comm', conn, if_exists='replace', index = True)
df_books.set_index('book_id').to_sql('books', conn, if_exists='replace', index = True)
conn.close()

In [3]:
#df_jaccard = pd.read_csv('../output/jacsim.csv')
df_jaccard = pd.read_csv('../analyse/distance_jaccard.csv')
# df_jaccard = df_jaccard.rename(columns = {'Unnamed: 0':'book_id'})
# df_jaccard = df_jaccard.set_index('book_id')

# df1 = df_jaccard.query('book_id == 1390851').transpose().dropna().squeeze()
# df2 = df_jaccard.loc[:,'1390851'].dropna()

# pd.concat([df1,df2], axis=0)

In [12]:
df_jaccard

,Unnamed: 0,905033,34397,869119,1700,643614,1455462,877839,8240,1022159,1258576,1162306,946624,8929,614802,151916,4392,7470,608790,831363,1212813,58718,905055,504720,12753,741161,577391,534727,716654,155768,23516,1071543,534632,663462,153516,146383,857108,833098,77097,744251,1064062,850282,1329810,721421,557588,18080,79250,1031310,1396553,830643,...,3072,408821,994347,825688,488289,927362,527000,13140,22615,912572,850399,855482,1140111,106693,352809,1120091,532707,717851,1128082,1114735,3456,230307,7613,19326,792892,955937,10781,38158,572563,1116212,5103,6529,7167,874240,1131235,490122,117001,104388,865852,1358259,956505,529204,1329550,940817,646574,8697,611806,891791,889201,609937
0,905033,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34397,0.862069,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,869119,0.850000,0.912281,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1700,0.844828,0.944444,0.894737,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,643614,0.680556,0.750000,0.718310,0.761194,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4561,8697,0.864407,0.894737,0.850000,0.877193,0.728571,0.761194,0.790323,0.915254,0.708333,0.877193,0.862069,0.781250,0.879310,0.892857,0.868852,0.862069,0.827586,0.857143,0.803279,0.456140,0.892857,0.816667,0.877193,0.862069,0.862069,0.827586,0.879310,0.830508,0.809524,0.800000,0.864407,0.890909,0.875000,0.818182,0.819672,0.841270,0.894737,0.875000,0.852459,0.819672,0.791045,0.847458,0.877193,0.866667,0.910714,0.803279,0.666667,0.676056,0.793103,...,0.877193,0.617284,0.723077,0.847458,0.836066,0.728571,0.857143,0.872727,0.813559,0.675676,0.762712,0.866667,0.774194,0.881356,0.816667,0.912281,0.890909,0.813559,0.718310,0.714286,0.830508,0.816667,0.790323,0.877193,0.857143,0.710145,0.852459,0.850000,0.946429,0.769231,0.662338,0.877193,0.872727,0.847458,0.614458,0.784615,0.830

In [20]:
# df_books[df_books['title'].str.contains('gang', case=False)]

# Preprocess pivot users

In [4]:
df_comm = pd.read_json('../output/final/data-comm-light.json',lines=True)
df_books = pd.read_json('../output/final/data-books.json',lines=True)

In [6]:
df_comm = df_comm[['book_id','user_id','note']]
df_comm['note'] = df_comm['note'].replace('',4.)
df_comm['note'] = df_comm['note'].astype(float)

In [7]:
pivot = df_comm.pivot_table(index='book_id', columns='user_id', values='note', aggfunc=np.sum, fill_value=0.)

In [54]:
pivot = pivot.applymap(lambda x: 1 if x >= 4 else 0)
# pas de sens de faire par note, c'est plutôt aimé ou pas aimé en commun. simcos de 0 est 0, et plus il y aura de 1

In [66]:
cs = cosine_similarity(pivot)

In [67]:
cs

array([[1.        , 0.1294996 , 0.02400809, ..., 0.16775366, 0.04987864,
        0.03178603],
       [0.1294996 , 1.        , 0.03400655, ..., 0.09401145, 0.02307857,
        0.01715843],
       [0.02400809, 0.03400655, 1.        , ..., 0.03064752, 0.        ,
        0.        ],
       ...,
       [0.16775366, 0.09401145, 0.03064752, ..., 1.        , 0.06482953,
        0.03670567],
       [0.04987864, 0.02307857, 0.        , ..., 0.06482953, 1.        ,
        0.        ],
       [0.03178603, 0.01715843, 0.        , ..., 0.03670567, 0.        ,
        1.        ]])

# Simple Model

In [55]:
class BookReco:
    def __init__(self):
        self.data = None
        self.vectors = []
        self.scalars = []
        self.users_cs = None
        self.predict_scores = None
        self.weights = {}
        self.df_jaccard = None
        self.df_users = None
        self.data_empty = None
        self.data_score = None

    def add_vector(self, col_prefix, weight=1):
        self.weights = {**self.weights, col_prefix: weight}
        self.vectors.append({'col_name': col_prefix})

    def add_jaccard(self, df, weight=1):
        self.weights = {**self.weights, 'jaccard': weight}
        self.df_jaccard = df

    def add_scalar(self, col_name, weight=1):
        self.weights = {**self.weights, col_name: weight}
        self.scalars.append({'col_name': col_name})

    def add_users(self, df_users, weight=1):
        self.weights = {**self.weights, 'users': weight}
        self.df_users = df_users

    def __get_cosine_similarity(self, prefix='sen_', data=None):
        if data is None:
            data_temp = self.data.filter(regex=f'^{prefix}',axis=1)
            data_temp = data_temp.fillna(0)
        else: data_temp = data
        vec = MinMaxScaler().fit_transform(data_temp)
        return cosine_similarity(vec)

    def fit(self, data):
        self.data = data
        self.data_empty = self.data[['book_id']]

        for i,vector in enumerate(self.vectors):
            feats_cs = self.__get_cosine_similarity(vector['col_name'])
            self.vectors[i] = {**vector, 'cosine_similar': feats_cs}

        for i,scalar in enumerate(self.scalars):
            X = self.data.loc[:,[scalar['col_name']]]
            X = MinMaxScaler().fit_transform(X)
            self.scalars[i] = {**scalar, 'scaled': X.reshape(-1)}

        if self.df_users is not None:
            self.users_cs = self.__get_cosine_similarity(prefix=None, data=self.df_users)

        del self.data # to reduce the size of the model. No need anymore the data

    def set_weight(self, weights:dict):
        self.weights = weights

    def __get_jaccard_score(self, book_id):
        df_jaccard = self.df_jaccard.copy()
        df_jaccard = df_jaccard.rename(columns = {'Unnamed: 0':'book_id'})
        df_jaccard = df_jaccard.set_index('book_id')

        try:
            df1 = df_jaccard.query('book_id == @book_id').transpose().dropna().squeeze()
            df2 = df_jaccard.loc[:, str(book_id)].dropna()
        except:
            print('Error Jaccard in __get_jaccard_score')
            return None

        if type(df1) is not pd.Series:
            return df2
        elif type(df2) is not pd.Series:
            return df1
        else:
            return pd.concat([df1,df2], axis=0)

    def predict(self, book_id):

        if self.data_empty is None:
            print(f"Fit the model before predict")
            return None
        
        try:
            index_book = self.data_empty.query('book_id == @book_id').index.values.astype(int)[0]
        except:
            print(f"Can't find book_id: {book_id} in the dataset")
            return None

        self.data_score = self.data_empty

        # get all scores, apply weight
        weight_sum = 0 # to normalize at the end, like a mean
        for vector in self.vectors:
            score = vector['cosine_similar'][index_book]
            weight = self.weights[vector['col_name']]
            weight_sum += weight

            self.data_score = pd.concat([self.data_score, pd.Series(score*weight, name=vector['col_name'])], axis=1)
        
        for scalar in self.scalars:
            weight = self.weights[scalar['col_name']]
            weight_sum += weight
            self.data_score = pd.concat([self.data_score, pd.Series(scalar['scaled']*weight, name=scalar['col_name'])], axis=1)

        self.data_score = self.data_score.set_index('book_id')
        
        if self.df_jaccard is not None:
            score = self.__get_jaccard_score(book_id) 

            if score is not None:
                score = score * self.weights['jaccard']
                weight_sum += self.weights['jaccard']
                self.data_score = self.data_score.merge(pd.Series(score, name='jaccard'), how='left', left_index=True, right_index=True)

        if self.users_cs is not None:
            score = self.users_cs[index_book]

            if score is not None:
                score = score * self.weights['users']
                weight_sum += self.weights['users']
                self.data_score = pd.concat([self.data_score, pd.Series(score, name='users')], axis=1)


        if weight_sum == 0:
            weight_sum == 1

        self.data_score =  self.data_score.sum(axis=1) / weight_sum

        #data = self.data.merge(pd.Series(self.data_score, name='score'), how='left', left_on='book_id', right_index=True)
        #return data.query('book_id != @book_id') # we pull off the asked book_id drom the scores
        return pd.Series(self.data_score[self.data_score.index != book_id], name='score')

    def format_predict(self, scores, max_books=5):
        scores = scores.sort_values(ascending=False)
        return scores[:max_books]

    def format_tojson(self, scores, data, max_books=5):
        data = data.copy().merge(scores, how='left', left_on='book_id', right_index=True)
        data = data.sort_values('score', ascending=False)

        output = []
        num = 0
        for _, book in data.iterrows():
            output.append({
                'title' : book['title'],
                'url' : book['book_url'],
                'image' : book['img_url'],
                'author' : book['surname']+' '+book['name'],
                'author_url' : book['book_author_url'],
                'book_rating_value' : book['book_rating_value'],
                'book_rating_count' : book['book_rating_count'],
                'score' : str(book['score']),
            })
            if num == max_books - 1:
                break
            num += 1
        return output


In [60]:
import modelreco

br = modelreco.BookReco()
# br = BookReco()
br.add_vector('tag_', weight=0)
br.add_vector('sen_', weight=50)
br.add_users(df_users=pivot, weight=0)
br.add_jaccard(df_jaccard, weight=50)
br.add_scalar('book_rating_value', weight=10)
br.add_scalar('book_nb_comm', weight=0)
br.add_scalar('book_rating_count', weight=0)
br.fit(df_books)

# br.set_weight({'tag_': 0.5, 'sen_': 1, 'jaccard': 1, 'book_rating_value': 1, 'book_nb_comm': 0})
# scores = br.predict(123961) # De Cape et de Crocs, tome 2 : Pavillon noir !
# br.format_tojson(scores, max_books=5)

In [59]:
br.set_weight({'tag_': 0, 'sen_': 0, 'users': 1, 'jaccard': 0, 'book_rating_value': 0, 'book_nb_comm': 0, 'book_rating_count': 0})
scores = br.predict(934945) # Germinal
#br.format_predict(scores, max_books=10)
br.format_tojson(scores, df_books, max_books=10)

[{'title': "L'histoire sans fin",
  'url': 'https://www.babelio.com/livres/Ende-Lhistoire-sans-fin/3818',
  'image': '/couv/CVT_CVT_LHistoire-sans-fin_9219.jpg',
  'author': 'Ende Michael',
  'author_url': '/auteur/Michael-Ende/3981',
  'book_rating_value': 4.16,
  'book_rating_count': 844,
  'score': '0.25954954703479527'},
 {'title': 'À la recherche du temps perdu - La Pléiade : Intégrale Tomes 1 et 2',
  'url': 'https://www.babelio.com/livres/Proust--la-recherche-du-temps-perdu-La-Pleiade--Integ/1655',
  'image': 'https://m.media-amazon.com/images/I/41iQ37pZhoL._SX195_.jpg',
  'author': 'Proust Marcel',
  'author_url': '/auteur/Marcel-Proust/2103',
  'book_rating_value': 4.59,
  'book_rating_count': 638,
  'score': '0.12309149097933271'},
 {'title': 'La Valse aux adieux',
  'url': 'https://www.babelio.com/livres/Kundera-La-Valse-aux-adieux/1675',
  'image': '/couv/CVT_La-Valse-aux-adieux_2902.jpg',
  'author': 'Kundera Milan',
  'author_url': '/auteur/Milan-Kundera/2129',
  'book_ra

In [61]:
import pickle
import dill

# warning, generate br only with modelreco.BookReco()
with open('../output/final/model-reco.obj', "wb") as f:
    dill.dump(br, f)

# with open('data/model-reco.obj', 'wb') as f:
#     pickle.dump(br, f, pickle.HIGHEST_PROTOCOL)

In [67]:
# for api and dashboard
conn = sqlite3.connect('../output/final/data.db')
sql_query = pd.read_sql(f'SELECT * FROM books WHERE book_id="{905033}"', conn)
dfb = pd.DataFrame(sql_query)
sql_query = pd.read_sql(f'SELECT * FROM comm WHERE book_id="{905033}"', conn)
dfc = pd.DataFrame(sql_query)
conn.close()
dfc

,book_id,book_nb_comm,title,name,surname,img_url,comm_id,user_id,note,date,appreciations,commentaire,gender,book_url,tags,book_rating_count,book_rating_value,book_author_url,book_editor,book_pages,sen_colère,sen_confiance,sen_honte,sen_joie,sen_peur,sen_réflexion,sen_surprise,sen_tristesse
0,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2080694,706958,4.0,30 novembre 2019,85,,,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
1,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1271915,319761,5.0,03 mars 2017,68,,F,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
2,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,1093854,128814,5.0,27 juin 2016,67,,M,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
3,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2711430,227017,4.5,22 juillet 2021,64,,M,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
4,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2685514,138438,4.5,26 juin 2021,64,,F,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2617422,954893,4.0,27 avril 2021,10,,,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
66,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2205523,723369,4.0,08 avril 2020,10,,,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
67,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2118642,723888,4.0,12 janvier 2020,10,,,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
68,905033,137,"Le gang de la clef à molette (Ne meurs pas, ô ...",Edward,Abbey,/couv/CVT_cvt_Le-gang-de-la-clef-a-molette_345...,2040408,769435,4.0,17 octobre 2019,10,,F,https://www.babelio.com/livres/Abbey-Le-gang-d...,"[['roman', 22], ['romans policiers et polars',...",678,3.96,/auteur/Edward-Abbey/23782,Gallmeister,491,0.211501,0.140185,0.009778,0.278578,0.077858,0.102662,0.069566,0.109873
